In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
classifier = Sequential()
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(64, (3, 3),  activation = 'tanh'))
classifier.add(Convolution2D(64, (3, 3),  activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(128, (3, 3),  activation = 'tanh'))
classifier.add(Convolution2D(128, (3, 3),  activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(activation = 'relu', units = 512))
classifier.add(Dropout(0.2))
classifier.add(Dense(activation = 'relu', units = 256))
classifier.add(Dropout(0.2))
classifier.add(Dense(activation = 'softmax', units = 2))

In [3]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
training_set = train_datagen.flow_from_directory(
        r'F:\Users\7\Desktop\dataset - sexo\train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        r'F:\Users\7\Desktop\dataset - sexo\test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 16487 images belonging to 2 classes.
Found 4123 images belonging to 2 classes.


In [6]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=16487//32,
        epochs=30,
        validation_data=test_set,
        validation_steps=4123//32)

Epoch 1/30
515/515 [==============================] - 33s 65ms/step - loss: 0.3929 - acc: 0.8192 - val_loss: 0.2689 - val_acc: 0.8896
Epoch 2/30
515/515 [==============================] - 31s 61ms/step - loss: 0.2573 - acc: 0.8974 - val_loss: 0.2842 - val_acc: 0.8927
Epoch 3/30
515/515 [==============================] - 31s 61ms/step - loss: 0.2275 - acc: 0.9094 - val_loss: 0.2145 - val_acc: 0.9176
Epoch 4/30
515/515 [==============================] - 31s 61ms/step - loss: 0.2005 - acc: 0.9216 - val_loss: 0.2289 - val_acc: 0.9127
Epoch 5/30
515/515 [==============================] - 31s 61ms/step - loss: 0.1897 - acc: 0.9286 - val_loss: 0.2218 - val_acc: 0.9240
Epoch 6/30
515/515 [==============================] - 31s 60ms/step - loss: 0.1835 - acc: 0.9309 - val_loss: 0.2484 - val_acc: 0.9005
Epoch 7/30
515/515 [==============================] - 31s 60ms/step - loss: 0.1739 - acc: 0.9348 - val_loss: 0.1969 - val_acc: 0.9237
Epoch 8/30
515/515 [==============================] - 31s 61ms

In [7]:
import pandas as pd
import numpy as np
predict_gen = test_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\test',
                                                              target_size=[64,64],
                                                              batch_size=4123,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = classifier.predict(X_val_sample)
#classifier.evaluate_generator(test_datagen, steps=1, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(onedtrue, onedpred))
from sklearn.metrics import classification_report
target_names = ['afr', 'afr-non']
print(classification_report(onedtrue, onedpred, target_names=target_names))

Found 4123 images belonging to 2 classes.
[[1777  158]
 [ 168 2020]]
              precision    recall  f1-score   support

         afr       0.91      0.92      0.92      1935
     afr-non       0.93      0.92      0.93      2188

   micro avg       0.92      0.92      0.92      4123
   macro avg       0.92      0.92      0.92      4123
weighted avg       0.92      0.92      0.92      4123



In [15]:
classifier.save('mf-92-basic.h5')

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout

vgg16 = VGG16(weights='imagenet')
x = vgg16.get_layer('fc1').output
x = BatchNormalization(name = 'BatchNormalization_0')(x)
x = Dropout(0.3, name = 'dropout0')(x)
x = vgg16.get_layer('fc2')(x)
x = BatchNormalization(name = 'BatchNormalization_1')(x)
x = Dropout(0.3, name = 'dropout1')(x)
prediction = Dense(output_dim=2, activation='softmax', name='prediction')(x)
model = Model(input=vgg16.input, output=prediction)

553467904/553467096 [==============================] - 1757s 3us/step


C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", name="prediction", units=2)`
  if sys.path[0] == '':
C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  del sys.path[0]


In [3]:
import pandas as pd
for layer in model.layers:
    if layer.name in ['fc1', 'fc2', 'prediction', 'dropout0', 'dropout1', 'flatten',
                      'block5_pool', 'block5_conv3', 'block5_conv2', 'block5_conv1',
                      'block4_pool', 'block4_conv3', 'block4_conv2',
                      'BatchNormalization_0', 'BatchNormalization_1']:
        continue
    layer.trainable = False

df = pd.DataFrame(([layer.name, layer.trainable] for layer in model.layers), columns=['layer', 'trainable'])
df.style.applymap(lambda trainable: f'background-color: {"yellow" if trainable else "red"}', subset=['trainable'])

,layer,trainable
0,input_1,False
1,block1_conv1,False
2,block1_conv2,False
3,block1_pool,False
4,block2_conv1,False
5,block2_conv2,False
6,block2_pool,False
7,block3_conv1,False
8,block3_conv2,False
9,block3_conv3,False


In [4]:
from keras.optimizers import SGD, Adam

sgd = SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):

    from keras.applications.vgg16 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\train',
                                                    target_size=[224,224],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\test',
                                                              target_size=[224,224],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 16487 images belonging to 2 classes.
Found 4123 images belonging to 2 classes.


In [6]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=16487//32,
                    nb_epoch=20,
                    validation_data=validation_generator,
                    validation_steps=4123//32)

C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=515, validation_data=<keras_pre..., validation_steps=128, epochs=20)`
  """


Epoch 1/20
515/515 [==============================] - 237s 461ms/step - loss: 0.4576 - acc: 0.8214 - val_loss: 0.2299 - val_acc: 0.9194
Epoch 2/20
515/515 [==============================] - 234s 455ms/step - loss: 0.3189 - acc: 0.8813 - val_loss: 0.2446 - val_acc: 0.9164
Epoch 3/20
515/515 [==============================] - 233s 452ms/step - loss: 0.2681 - acc: 0.9027 - val_loss: 0.3401 - val_acc: 0.8829
Epoch 4/20
515/515 [==============================] - 234s 455ms/step - loss: 0.2279 - acc: 0.9167 - val_loss: 0.3170 - val_acc: 0.8866
Epoch 5/20
515/515 [==============================] - 234s 454ms/step - loss: 0.2197 - acc: 0.9194 - val_loss: 0.1971 - val_acc: 0.9374
Epoch 6/20
515/515 [==============================] - 238s 461ms/step - loss: 0.1913 - acc: 0.9301 - val_loss: 0.1971 - val_acc: 0.9279
Epoch 7/20
515/515 [==============================] - 230s 446ms/step - loss: 0.1814 - acc: 0.9328 - val_loss: 0.2027 - val_acc: 0.9298
Epoch 8/20
515/515 [============================

In [8]:
import pandas as pd
import numpy as np
predict_gen = validation_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\test',
                                                              target_size=[224,224],
                                                              batch_size=4123,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)
#model.evaluate_generator(validation_datagen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(onedtrue, onedpred))
from sklearn.metrics import classification_report
target_names = ['fem', 'masc']
print(classification_report(onedtrue, onedpred, target_names=target_names))

Found 4123 images belonging to 2 classes.
[[1672  263]
 [  59 2129]]
              precision    recall  f1-score   support

         fem       0.97      0.86      0.91      1935
        masc       0.89      0.97      0.93      2188

   micro avg       0.92      0.92      0.92      4123
   macro avg       0.93      0.92      0.92      4123
weighted avg       0.93      0.92      0.92      4123



In [10]:
model.save('mf-92-vgg16.h5')

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_icpt(x):

    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_icpt,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_icpt)
validation_generator = validation_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 16487 images belonging to 2 classes.
Found 4123 images belonging to 2 classes.


In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
inc_model = InceptionV3(weights='imagenet', include_top=False)

# get layers and add average pooling layer
x = inc_model.output
x = GlobalAveragePooling2D()(x)

# add fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
# add output layer
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inc_model.input, outputs=predictions)

# freeze pre-trained model area's layer
for layer in inc_model.layers:
    layer.trainable = False

# update the weight that are added
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=1,
                    validation_data=validation_generator,
                    validation_steps=32)

# choose the layers which are updated by training
#layer_num = len(model.layers)
#for layer in model.layers[:248]:
#    layer.trainable = False

for layer in model.layers:
    layer.trainable = True

# training
model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, validation_data=<keras_pre..., validation_steps=32, epochs=1)`


Epoch 1/1
64/64 [==============================] - 21s 324ms/step - loss: 1.1451 - acc: 0.5610 - val_loss: 0.5827 - val_acc: 0.6953


In [6]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=16487//32,
                    nb_epoch=30,
                    shuffle = True,
                    validation_data=validation_generator,
                    validation_steps=4123//32)

C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\luido\AppData\Local\Continuum\anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=515, shuffle=True, validation_data=<keras_pre..., validation_steps=128, epochs=30)`
  


Epoch 1/30
515/515 [==============================] - 115s 224ms/step - loss: 0.1438 - acc: 0.9470 - val_loss: 0.1859 - val_acc: 0.9336
Epoch 2/30
515/515 [==============================] - 114s 222ms/step - loss: 0.1427 - acc: 0.9489 - val_loss: 0.1861 - val_acc: 0.9357
Epoch 3/30
515/515 [==============================] - 114s 221ms/step - loss: 0.1415 - acc: 0.9495 - val_loss: 0.1846 - val_acc: 0.9328
Epoch 4/30
515/515 [==============================] - 121s 236ms/step - loss: 0.1391 - acc: 0.9503 - val_loss: 0.1845 - val_acc: 0.9345
Epoch 5/30
515/515 [==============================] - 118s 228ms/step - loss: 0.1366 - acc: 0.9506 - val_loss: 0.1775 - val_acc: 0.9389
Epoch 6/30
515/515 [==============================] - 114s 221ms/step - loss: 0.1313 - acc: 0.9534 - val_loss: 0.1840 - val_acc: 0.9396
Epoch 7/30
515/515 [==============================] - 112s 218ms/step - loss: 0.1323 - acc: 0.9529 - val_loss: 0.1820 - val_acc: 0.9360
Epoch 8/30
515/515 [============================

In [7]:
import pandas as pd
import numpy as np
predict_gen = validation_datagen.flow_from_directory(directory=r'F:\Users\7\Desktop\dataset - sexo\test',
                                                              target_size=[140,140],
                                                              batch_size=4123,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)
#model.evaluate_generator(validation_datagen, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(onedtrue, onedpred))
from sklearn.metrics import classification_report
target_names = ['fem', 'masc']
print(classification_report(onedtrue, onedpred, target_names=target_names))

Found 4123 images belonging to 2 classes.
[[1825  110]
 [ 132 2056]]
              precision    recall  f1-score   support

         fem       0.93      0.94      0.94      1935
        masc       0.95      0.94      0.94      2188

   micro avg       0.94      0.94      0.94      4123
   macro avg       0.94      0.94      0.94      4123
weighted avg       0.94      0.94      0.94      4123



In [8]:
model.save('mf-94-incept')